In [14]:
# Imports
import numpy as np
import json

# Dados
# Considere os 8 pontos 3D abaixo formando um cubo de aresta 1:
points = [
    [0, 1, 1], # P0
    [1, 1, 1], # P1
    [1, 0, 1], # P2
    [0, 0, 1], # P3
    [1, 1, 0], # P4
    [1, 0, 0], # P5
    [0, 1, 0], # P6
    [0, 0, 0], # P7
]

# (i )As coordenadas dos pixels correspondentes $\mathbf{u_{nk}}$ para $n=0,...,5$ e $k=0,...,7$)
u_nk = np.zeros((6,8,2))
qui_nk = np.zeros((6,8), dtype=int)

# Para a câmera 0: 
P_visados = [0, 1, 2, 3]
qui_nk[0, P_visados] = 1
u_nk[0,0] = [255,144]
u_nk[0,1] = [781,114]
u_nk[0,2] = [781,640]
u_nk[0,3] = [255,640]

# Para a câmera 1:
P_visados = [0, 1, 2, 3, 4, 5]
qui_nk[1, P_visados] = 1
u_nk[1,0] = [215,176]
u_nk[1,1] = [620,86]
u_nk[1,2] = [625,703]
u_nk[1,3] = [235,595]
u_nk[1,4] = [749,196]
u_nk[1,5] = [748,540]

# Para a câmera 2: 
P_visados = [0, 1, 2, 3, 4, 5]
qui_nk[2, P_visados] = 1
u_nk[2,0] = [294,240]
u_nk[2,1] = [375,208]
u_nk[2,2] = [434,660]
u_nk[2,3] = [337,522]
u_nk[2,4] = [745,187]
u_nk[2,5] = [755,541]

# Para a câmera 3: 
P_visados = [1, 2, 4, 5, 6, 7]
qui_nk[3, P_visados] = 1
u_nk[3,1] = [256,246]
u_nk[3,2] = [287,581]
u_nk[3,4] = [526,106]
u_nk[3,5] = [598,565]
u_nk[3,6] = [685,215]
u_nk[3,7] = [743,508]

# Para a câmera 4: 
P_visados = [1, 2, 4, 5, 6, 7, 3]
qui_nk[4, P_visados] = 1
u_nk[4,1] = [338,287]
u_nk[4,2] = [272,541]
u_nk[4,4] = [448,86]
u_nk[4,5] = [388,407]
u_nk[4,6] = [724,233]
u_nk[4,7] = [767,525]
u_nk[4,3] = [550,600]

# Para a câmera 5:
P_visados = [4, 5, 6, 7, 0, 3]
qui_nk[5, P_visados] = 1
u_nk[5,4] = [333,154]
u_nk[5,5] = [265,460]
u_nk[5,6] = [571,215]
u_nk[5,7] = [440,649]
u_nk[5,0] = [783,290]
u_nk[5,3] = [673,578]


qui_nk_expanded = np.repeat(qui_nk, 4, axis=1)
qui_nk_expanded = np.repeat(qui_nk_expanded, 2, axis=0)

print(qui_nk)
k = 7
print(qui_nk_expanded[:,4*(k):4*(k+1)]) 


# (ii) As respectivas matrizes $T_n$, para $n=0,...,5$ e $K$ são:

# Load the jsons
cameras = []
for i in range(1,7):
    cameras.append(json.load(open(f"aux_data/CuboCamera{i}.json")))

# Load the Tn
T = []
for i in range(0,6):
    T.append(np.array(cameras[i]['extrinsic'], dtype=float).reshape(4,4).T)
# Load K
K = []
for i in range(0,6):
    K.append(np.array(cameras[i]['intrinsic']['intrinsic_matrix'], dtype=float).reshape(3,3).T)

# show Ts
for i in range(0,6):
    print(f"T{i}:\n", T[i])

# show Ks or K
def all_matrices_are_close(matrix_list, rtol=1e-5, atol=1e-8):
    reference = matrix_list[0]
    return all(np.allclose(reference, m, rtol=rtol, atol=atol) for m in matrix_list[1:])

if all_matrices_are_close(K):
    K = K[0]
    print("K:\n", K)
else:
    for i in range(0,6):
        print(f"K{i}:\n", K[i])

# (iii) As matrizes de projeção $P_n$, para $n=0,...,5$ e $K$ são:

# Build Projection matrix P
P = []
for i in range(0,6):
    P.append(K @ T[i][:3,:])
# show Ps
for i in range(0,6):
    print(f"P{i}:\n", P[i])

# (iv) Ajustamento

results = []
esperado = np.array(points)
for k in range(0,8):
    # Ajustamento do ponto Pk em todas as câmeras
    Ak = []
    for n in range(0,6):
        Pn_1 = P[n][0]
        Pn_2 = P[n][1]
        Pn_3 = P[n][2]
        u, v = u_nk[n,k]
        if len(Ak) == 0:
            Ak = np.array([u * Pn_3 - Pn_1,
                            v * Pn_3 - Pn_2,], dtype=float)
        else: 
            aux = np.array([u * Pn_3 - Pn_1,
                            v * Pn_3 - Pn_2], dtype=float)
            Ak = np.vstack((Ak, aux))

    # print("A.shape", Ak.shape, f" / Esperado (2Nx4) = {2*6 }x{4*1}")

    # Adicionando o relê Qui 
    q = qui_nk_expanded[:,4*(k):4*(k+1)]
    Ak = np.multiply(q, Ak)

    # # Compute eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(Ak.T @ Ak)

    # # # Find the index of the minimum eigenvalue
    min_index = np.argmin(eigenvalues)

    # # # Get the minimum eigenvalue and corresponding eigenvector
    min_eigenvalue = eigenvalues[min_index]
    min_eigenvector = eigenvectors[:, min_index]

    # # Remove the imaginary part if exists
    min_eigenvector = np.real(min_eigenvector)

    P_calc_homo = min_eigenvector/min_eigenvector[-1] # Normalizando o vetor
    P_calc_homo = np.round(P_calc_homo, 5)
    P_calc = P_calc_homo[0:3]
    P_esperado = esperado[k]
    print(f"P{k}:", P_calc, "--> Erro:", np.linalg.norm(P_esperado-P_calc))
    results.append(P_calc)

erro = np.linalg.norm(esperado-results)
print("Erro total:", erro)

# (v) Reprojeção e seleção de outliers
results_homogeneus = np.hstack((np.array(results) , np.ones((8,1))))
tol = 6
outliers_nk = []
for n in range(0,6):
    for k in range(0,8):
        if qui_nk[n,k]:
            X_k = results_homogeneus[k]
            u_nk_predito_homogeneus = P[n] @ X_k
            u_nk_predito = u_nk_predito_homogeneus/u_nk_predito_homogeneus[-1]
            u_nk_predito = u_nk_predito[:-1] # remove a coordenada homogenea
            erro = np.linalg.norm(u_nk_predito -  u_nk[n,k])
            if erro>tol: 
                outliers_nk.append((n,k)) 
            print(n, k, u_nk_predito,  u_nk[n,k], erro, "--> outlier" if erro>tol else "")

[[1 1 1 1 0 0 0 0]
 [1 1 1 1 1 1 0 0]
 [1 1 1 1 1 1 0 0]
 [0 1 1 0 1 1 1 1]
 [0 1 1 1 1 1 1 1]
 [1 0 0 1 1 1 1 1]]
[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]
T0:
 [[ 1.          0.          0.         -0.488     ]
 [-0.         -1.         -0.          0.488     ]
 [-0.         -0.         -1.          2.26162003]
 [ 0.          0.          0.          1.        ]]
T1:
 [[ 0.86082046 -0.01883655 -0.50856004 -0.16271085]
 [ 0.02006227 -0.99728187  0.07089696  0.44228545]
 [-0.50851316 -0.07123242 -0.85810274  2.47529001]
 [ 0.          0.          0.          1.        ]]
T2:
 [[ 0.45223409 -0.09387435 -0.88694528  0.25794975]
 [ 0.12825264 -0.97726591  0.16882712  0.33193084]
 [-0.88262993 -0.19010245 -0.42991333  2.82616274]
 [ 0.          0.          0.          1.        ]]
T3:
 [[-0.56821747 -0.14256521 -0.81043449  0.74235398]
 [-0.03496948 -0.97980427  0.19687744  0.3991334 ]
 [-0.82213505  

In [15]:
# (vi) Considerar outliers como ponto não visualizados na imagem
for el in outliers_nk:
    qui_nk[el] = 0

qui_nk_expanded = np.repeat(qui_nk, 4, axis=1)
qui_nk_expanded = np.repeat(qui_nk_expanded, 2, axis=0)

k = 0 # Esse ponto 3D 0 não tem pixel em nenhuma imagem
print(qui_nk_expanded[:,4*(k):4*(k+1)])

# (vii) usar os demais pontos para refazer o ajustametno

results = []
esperado = np.array(points)
for k in range(0,8):
    # Ajustamento do ponto Pk em todas as câmeras
    Ak = []
    for n in range(0,6):
        Pn_1 = P[n][0]
        Pn_2 = P[n][1]
        Pn_3 = P[n][2]
        u, v = u_nk[n,k]
        if len(Ak) == 0:
            Ak = np.array([u * Pn_3 - Pn_1,
                            v * Pn_3 - Pn_2,], dtype=float)
        else: 
            aux = np.array([u * Pn_3 - Pn_1,
                            v * Pn_3 - Pn_2], dtype=float)
            Ak = np.vstack((Ak, aux))

    # print("A.shape", Ak.shape, f" / Esperado (2Nx4) = {2*6 }x{4*1}")

    # Adicionando o relê Qui 
    q = qui_nk_expanded[:,4*(k):4*(k+1)]
    Ak = np.multiply(q, Ak)

    # # Compute eigenvalues and eigenvectors
    eigenvalues, eigenvectors = np.linalg.eig(Ak.T @ Ak)

    # # # Find the index of the minimum eigenvalue
    min_index = np.argmin(eigenvalues)

    # # # Get the minimum eigenvalue and corresponding eigenvector
    min_eigenvalue = eigenvalues[min_index]
    min_eigenvector = eigenvectors[:, min_index]

    # # Remove the imaginary part if exists
    min_eigenvector = np.real(min_eigenvector)

    P_calc_homo = min_eigenvector/min_eigenvector[-1] # Normalizando o vetor
    P_calc_homo = np.round(P_calc_homo, 5)
    P_calc = P_calc_homo[0:3]
    P_esperado = esperado[k]
    print(f"P{k}:", P_calc, "--> Erro:", np.linalg.norm(P_esperado-P_calc))
    results.append(P_calc)

erro = np.linalg.norm(esperado-results)
print("Erro total:", erro)

[[0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [1 1 1 1]
 [1 1 1 1]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [0 0 0 0]
 [1 1 1 1]
 [1 1 1 1]]
P0: [-3.00000e-04  1.00034e+00  9.98530e-01] --> Erro: 0.0015383432646844204
P1: [0.99977 1.00016 1.00031] --> Erro: 0.00041785164831549776
P2: [9.9997e-01 2.1000e-04 9.9985e-01] --> Erro: 0.0002598076211353191
P3: [6.00000e-04 7.50000e-04 1.00051e+00] --> Erro: 0.0010874741376235163
P4: [ 9.98620e-01  1.00067e+00 -4.00000e-05] --> Erro: 0.0015345683432157929
P5: [ 1.00045e+00 -2.80000e-04 -2.00000e-04] --> Erro: 0.0005664803615307906
P6: [-6.20000e-04  1.00042e+00 -2.00000e-05] --> Erro: 0.0007491328320131705
P7: [-0.00326 -0.00278 -0.00376] --> Erro: 0.005700315780726537
Erro total: 0.006286636620642241
